In [41]:
import pandas as pd       
import os 
import math 
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt  
import re

In [19]:
os.chdir('/media/abhi/0B8B16F90B8B16F9/git-repos/accent-classification-deep-learning/recordings/recordings/')

In [9]:
ls

recordings/


In [49]:
SAMPLE_RATE = 24000

In [75]:
df = pd.DataFrame(columns=['filename','mfcc','mfcc_delta','mfcc_std','accent'])

In [76]:
row_list = []

In [77]:
prev_a = ''
for each in os.listdir():
    accent = re.split('\d',each)[0]
    if prev_a != accent:
        print(accent)
    else:
        prev_a = accent

afrikaans
afrikaans
afrikaans
afrikaans
afrikaans
agni
akan
albanian
albanian
albanian
albanian
albanian
albanian
albanian
albanian
albanian
amazigh
amazigh
amharic
amharic
amharic
amharic
amharic
amharic
amharic
amharic
amharic
amharic
amharic
amharic
amharic
amharic
amharic
amharic
amharic
amharic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
amharic
amharic
arabic
arabic
arabic
arabic
arabic
azerbaijani
belarusan
bengali
bulgarian
cantonese
catalan
czech
dari
dutch
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic
arabic

In [78]:
for each in os.listdir():
    accent = re.split('\d',each)[0]
#     print(accent)
    print(accent,each, end = "\t")
    waveform, sr = librosa.load(each, sr=SAMPLE_RATE)
    mfcc = librosa.feature.mfcc(waveform, sr=SAMPLE_RATE)
    mfcc_delta = np.mean(librosa.feature.delta(mfcc),axis=1)
    mfcc = np.mean(librosa.feature.mfcc(waveform,SAMPLE_RATE), axis=1)
    mfcc_std = np.std(librosa.feature.mfcc(waveform,SAMPLE_RATE), axis=1)
#     print(mfcc_std.shape)
    features = np.hstack([mfcc, mfcc_delta,mfcc_std])
    print(features.shape,mfcc_std.shape,mfcc_delta.shape,mfcc.shape)
#     print(mfcc_std)
#     print(mfcc_delta)
#     print(mfcc)
    row_list.append({'filename': each,'mfcc': mfcc,'mfcc_delta': mfcc_delta,'mfcc_std': mfcc_std,'accent': accent})
#     print(mfcc)
#     print(mfcc.shape)
#     plt.figure()
#     librosa.display.specshow(mfcc, x_axis='time')
#     print(df)
#     break

afrikaans afrikaans1.mp3	

/home/abhi/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


(60,) (20,) (20,) (20,)
afrikaans afrikaans2.mp3	

/home/abhi/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


(60,) (20,) (20,) (20,)
afrikaans afrikaans3.mp3	

...

(60,) (20,) (20,) (20,)
yupik yupik1.mp3	

/home/abhi/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


(60,) (20,) (20,) (20,)
zulu zulu1.mp3	

/home/abhi/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


(60,) (20,) (20,) (20,)


In [85]:
df = pd.DataFrame(row_list)       

In [86]:
df.describe()

,filename,mfcc,mfcc_delta,mfcc_std,accent
count,2138,2138,2138,2138,2138
unique,2138,2138,2138,2138,200
top,kurdish6.mp3,"[-301.3997, 61.878613, 0.15949716, 8.90255, -7...","[-0.079046585, 0.021570988, 0.012377683, -0.01...","[89.527, 62.44772, 29.414152, 26.379177, 22.64...",english
freq,1,1,1,1,579


In [87]:
df.to_pickle('../features.pkl')